In [1]:
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import src.data.breathe_data as bd
import src.data.helpers as dh
import src.inference.helpers as ih
import src.modelling_ar.ar as ar
import src.inf_cutset_conditioning.helpers as cutseth
import src.inf_cutset_conditioning.cutset_cond_algs_learn_ar_change as cca_ar_change
import src.models.helpers as mh

import json
import numpy as np

In [42]:
# Load data
df = bd.load_meas_from_excel("BR_O2_FEV1_FEF2575_conservative_smoothing_with_idx")

INFO:root:* Checking for same day measurements *


In [ ]:
id = "134"
dftmp, start_idx, end_idx = dh.find_longest_consec_series(df[df.ID == id], n_days=3)


ecfev1_noise_model_suffix = "_std_add_mult_fev1"
ar_prior = "breathe (2 days model, ecFEV1 addmultnoise, ecFEF25-75)"
ar_change_cpt_suffix = "_shape_factor1"

(
    # p_M_given_D,
    # log_p_D_given_M,
    # AR_given_M_and_D,
    # AR_given_M_and_all_D,
    log_p_S_given_D,
    res_dict,
) = cca_ar_change.run_long_noise_model_through_time(
    dftmp,
    ar_prior=ar_prior,
    ar_change_cpt_suffix=ar_change_cpt_suffix,
    ecfev1_noise_model_suffix=ecfev1_noise_model_suffix,
)

## Process results

In [18]:
def load_and_process_json(card_suffix, params, labels):
    path = f"{dh.get_path_to_src()}inf_cutset_conditioning/p_s_given_d_card{card_suffix}.json"

    # Load the JSON file
    with open(path, "r") as file:
        data = json.load(file)

    # Convert lists to NumPy arrays
    for key in data:
        data[key] = np.array(data[key])

    # Replace 'nan' with np.nan and "-inf" with -np.inf
    for key in data:
        data[key][np.where(data[key] == "-inf")] = -np.inf

    # REmove dtypes from data
    data = {key: data[key].astype(float) for key in data}

    # Remove entries with only nan values
    data = {key: data[key] for key in data if not np.all(np.isnan(data[key]))}

    log_p_s_given_D = np.sum(list(data.values()), axis=0)

    df_heatmap = pd.DataFrame(
        columns=["log_p_S_given_D", "params"], data=zip(log_p_s_given_D, params)
    )
    # Split params into three columns "main width", "tail width", "main weight"
    df_heatmap[labels] = pd.DataFrame(
        df_heatmap["params"].tolist(), index=df_heatmap.index
    )
    return df_heatmap, data

In [23]:
params = [
    (1, 10, 0.5),
    (1, 10, 0.7),
    (1, 10, 0.9),
    (1, 30, 0.5),
    (1, 30, 0.7),
    (1, 30, 0.9),
    (1, 50, 0.5),
    (1, 50, 0.7),
    (1, 50, 0.9),
    (3, 10, 0.5),
    (3, 10, 0.7),
    (3, 10, 0.9),
    (3, 30, 0.5),
    (3, 30, 0.7),
    (3, 30, 0.9),
    (3, 50, 0.5),
    (3, 50, 0.7),
    (3, 50, 0.9),
    (5, 10, 0.5),
    (5, 10, 0.7),
    (5, 10, 0.9),
    (5, 30, 0.5),
    (5, 30, 0.7),
    (5, 30, 0.9),
    (5, 50, 0.5),
    (5, 50, 0.7),
    (5, 50, 0.9),
]
labels = ["main width", "tail width", "main weight"]
df_heatmap, IDs = load_and_process_json(27, params, labels)

In [ ]:
# Plot heatmap with main width on x axis, tail width on y axis and log_p_S_given_D as the color, just use the first 3 main weights


def add_heatmap_for_weight(fig, df, weight, col):
    df = df[df["main weight"] == weight]
    fig.add_trace(
        go.Heatmap(z=df["log_p_S_given_D"], x=df["main width"], y=df["tail width"]),
        row=1,
        col=col,
    )
    fig.update_xaxes(
        type="category",
        row=1,
        col=col,
        title="Main width<br><br>Main weight=" + str(weight),
    )
    return fig


def plot_3_3_heatmaps(df_heatmap, IDs, w1, w2, w3):
    fig = make_subplots(1, 3)

    fig = add_heatmap_for_weight(fig, df_heatmap, w1, 1)
    fig = add_heatmap_for_weight(fig, df_heatmap, w2, 2)
    fig = add_heatmap_for_weight(fig, df_heatmap, w3, 3)

    fig.update_layout(
        title=f"Log probability of S given D for different AR change parameters ({len(IDs)} IDs)",
        yaxis_title="Tail width",
        # yaxis_type='log',
        width=650,
        height=300,
        font_size=10,
    )
    # Recude font size of the subplot titles
    for annotation in fig["layout"]["annotations"]:
        annotation["font"] = dict(size=14)
    # use the same color scale for all subplots
    fig.update_traces(
        zmin=min(df_heatmap.log_p_S_given_D[df_heatmap.log_p_S_given_D != -np.inf]),
        zmax=max(df_heatmap.log_p_S_given_D),
        colorbar=dict(title="log_p_S_given_D"),
    )
    # X axis and y axis are categorical
    fig.update_yaxes(type="category")

    fig.show()

In [ ]:
plot_3_3_heatmaps(df_heatmap, IDs, 0.5, 0.7, 0.9)

In [ ]:
params = [
    (1, 10, 0.7),
    (1, 10, 0.73),
    (1, 10, 0.76),
    (1, 10, 0.79),
    (1, 10, 0.81),
    (1, 10, 0.84),
    (1, 10, 0.87),
    (1, 10, 0.9),
    (1, 10, 0.93),
    (1, 10, 0.96),
    (1, 10, 0.99),
]

df_heatmap, IDs = load_and_process_json(11, params, labels)
df_heatmap

,log_p_S_given_D,params,main width,tail width,main weight
0,-3.990078e+06,"(1, 10, 0.7)",1,10,0.70
1,-3.989833e+06,"(1, 10, 0.73)",1,10,0.73
2,-3.989596e+06,"(1, 10, 0.76)",1,10,0.76
3,-3.989375e+06,"(1, 10, 0.79)",1,10,0.79
4,-3.989233e+06,"(1, 10, 0.81)",1,10,0.81
5,-3.989029e+06,"(1, 10, 0.84)",1,10,0.84
6,-3.988834e+06,"(1, 10, 0.87)",1,10,0.87
7,-3.988648e+06,"(1, 10, 0.9)",1,10,0.90
8,-3.988472e+06,"(1, 10, 0.93)",1,10,0.93
9,-3.988306e+06,"(1, 10, 0.96)",1,10,0.96


In [6]:
params = [
    (0.2, 1, 0.9),
    (0.2, 1, 0.95),
    (0.2, 1, 1),
    (0.2, 4, 0.9),
    (0.2, 4, 0.95),
    (0.2, 4, 1),
    (0.2, 10, 0.9),
    (0.2, 10, 0.95),
    (0.2, 10, 1),
    (0.4, 1, 0.9),
    (0.4, 1, 0.95),
    (0.4, 1, 1),
    (0.4, 4, 0.9),
    (0.4, 4, 0.95),
    (0.4, 4, 1),
    (0.4, 10, 0.9),
    (0.4, 10, 0.95),
    (0.4, 10, 1),
    (1, 1, 0.9),
    (1, 1, 0.95),
    (1, 1, 1),
    (1, 4, 0.9),
    (1, 4, 0.95),
    (1, 4, 1),
    (1, 10, 0.9),
    (1, 10, 0.95),
    (1, 10, 1),
]
labels = ["main width", "tail width", "main weight"]
df_heatmap, IDs = load_and_process_json(272, params, labels)
plot_3_3_heatmaps(df_heatmap, IDs, 0.9, 0.95, 1)

In [16]:
params = [
    (0.01, 0.1, 0.7),
    (0.01, 0.2, 0.7),
    (0.01, 0.5, 0.7),
    (0.01, 0.8, 0.7),
    (0.01, 1.1, 0.7),
    (0.01, 1.4, 0.7),
    (0.03, 0.1, 0.7),
    (0.03, 0.2, 0.7),
    (0.03, 0.5, 0.7),
    (0.03, 0.8, 0.7),
    (0.03, 1.1, 0.7),
    (0.03, 1.4, 0.7),
    (0.05, 0.1, 0.7),
    (0.05, 0.2, 0.7),
    (0.05, 0.5, 0.7),
    (0.05, 0.8, 0.7),
    (0.05, 1.1, 0.7),
    (0.05, 1.4, 0.7),
    (0.1, 0.2, 0.7),
    (0.1, 0.5, 0.7),
    (0.1, 0.8, 0.7),
    (0.1, 1.1, 0.7),
    (0.1, 1.4, 0.7),
    (0.15, 0.2, 0.7),
    (0.15, 0.5, 0.7),
    (0.15, 0.8, 0.7),
    (0.15, 1.1, 0.7),
    (0.15, 1.4, 0.7),
]
labels = ["main width", "tail width", "main weight"]
df_heatmap, IDs = load_and_process_json(28, params, labels)

fig = make_subplots()
fig = add_heatmap_for_weight(fig, df_heatmap, 0.7, 1)
fig.update_layout(
    title=f"Log probability of S given D for different AR change parameters ({len(IDs)} IDs)",
    yaxis_title="Tail width",
    # yaxis_type='log',
    width=450,
    height=400,
    font_size=10,
)
fig.show()

In [31]:
params = [
    (0.001, 0.01, 0.7),
    (0.001, 0.03, 0.7),
    (0.001, 0.07, 0.7),
    (0.001, 0.1, 0.7),
    (0.001, 0.15, 0.7),
    (0.003, 0.01, 0.7),
    (0.003, 0.03, 0.7),
    (0.003, 0.07, 0.7),
    (0.003, 0.1, 0.7),
    (0.003, 0.15, 0.7),
    (0.007, 0.01, 0.7),
    (0.007, 0.03, 0.7),
    (0.007, 0.07, 0.7),
    (0.007, 0.1, 0.7),
    (0.007, 0.15, 0.7),
    (0.01, 0.03, 0.7),
    (0.01, 0.07, 0.7),
    (0.01, 0.1, 0.7),
    (0.01, 0.15, 0.7),
    (0.025, 0.03, 0.7),
    (0.025, 0.07, 0.7),
    (0.025, 0.1, 0.7),
    (0.025, 0.15, 0.7),
]
labels = ["main width", "tail width", "main weight"]
df_heatmap, data = load_and_process_json(23, params, labels)

fig = make_subplots()
fig = add_heatmap_for_weight(fig, df_heatmap, 0.7, 1)
fig.update_layout(
    title=f"Log probability of S given D for different shape parameters<br>({len(IDs)} IDs)",
    yaxis_title="Tail width",
    # yaxis_type='log',
    width=450,
    height=400,
    font_size=10,
)
fig.show()

In [41]:
for id in data.keys():

    df_heatmap_for_ID = df_heatmap
    df_heatmap_for_ID['log_p_S_given_D'] = data[id]

    # Plot for ID
    fig = make_subplots()
    fig = add_heatmap_for_weight(fig, df_heatmap_for_ID, 0.7, 1)
    fig.update_layout(
        title=f"Log probability of S given D for different shape parameters",
        yaxis_title="Tail width",
        # yaxis_type='log',
        width=800,
        height=600,
        font_size=12,
    )
    fig.write_image(
        f"{dh.get_path_to_main()}PlotsBreathe/Interconnecting_ARs_entries/heatmaps_optimal_ar_change_factor_shape/ID {id}.pdf"
    )

In [44]:
df[df.ID == '248']

,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,ecFEV1,ecFEF2575,Sex,Height,Age,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,ecFEF2575%ecFEV1,idx ecFEV1 (L),idx O2 saturation (%),idx ecFEF2575%ecFEV1,idx ecFEF25-75 % ecFEV1 (%)
23944,248,2020-05-22,1.20,94,0.88,1.20,1.13,Female,154.0,44,2.601447,98.311859,46.128180,46.128180,95.614101,73.333333,24,44,36,36
23945,248,2020-09-14,1.13,96,0.91,1.13,0.91,Female,154.0,44,2.601447,98.311859,43.437370,43.437370,97.648444,80.530973,22,46,40,40
23946,248,2020-12-16,1.33,97,1.05,1.33,1.05,Female,154.0,44,2.601447,98.311859,51.125400,51.125400,98.665615,78.947368,26,47,39,39
23947,248,2021-04-07,1.16,99,0.84,1.16,1.05,Female,154.0,44,2.601447,98.311859,44.590574,44.590574,100.699957,72.413793,23,49,36,36
23948,248,2022-05-14,0.95,98,0.74,0.95,0.74,Female,154.0,44,2.601447,98.311859,36.518143,36.518143,99.682786,77.894737,19,48,38,38
23949,248,2022-05-16,1.06,98,0.76,1.06,0.76,Female,154.0,44,2.601447,98.311859,40.746559,40.746559,99.682786,71.698113,21,48,35,35
23950,248,2022-05-17,1.13,99,0.80,1.13,0.80,Female,154.0,44,2.601447,98.311859,43.437370,43.437370,100.699957,70.796460,22,49,35,35
23951,248,2022-05-19,1.12,98,0.80,1.12,0.80,Female,154.0,44,2.601447,98.311859,43.052968,43.052968,99.682786,71.428571,22,48,35,35
23952,248,2022-05-21,1.09,99,0.75,1.09,0.75,Female,154.0,44,2.601447,98.311859,41.899764,41.899764,100.699957,68.807339,21,49,34,34
23953,248,2022-05-22,0.90,99,0.65,0.90,0.65,Female,154.0,44,2.601447,98.311859,34.596135,34.596135,100.699957,72.222222,18,49,36,36


## Process results (archive)

In [148]:
# Element wise addition of the values in data
params = [
    (2, 15, 0.3),
    (2, 15, 0.5),
    (2, 15, 0.7),
    (2, 30, 0.3),
    (2, 30, 0.5),
    (2, 30, 0.7),
    (2, 50, 0.3),
    (2, 50, 0.5),
    (2, 50, 0.7),
    (5, 15, 0.3),
    (5, 15, 0.5),
    (5, 15, 0.7),
    (5, 30, 0.3),
    (5, 30, 0.5),
    (5, 30, 0.7),
    (5, 50, 0.3),
    (5, 50, 0.5),
    (5, 50, 0.7),
    (8, 15, 0.3),
    (8, 15, 0.5),
    (8, 15, 0.7),
    (8, 30, 0.3),
    (8, 30, 0.5),
    (8, 30, 0.7),
    (8, 50, 0.3),
    (8, 50, 0.5),
    (8, 50, 0.7),
]
labels = ["main width", "tail width", "main weight"]

df_heatmap, IDs = load_and_process_json(27, params, labels)
df_heatmap

,log_p_S_given_D,params,main width,tail width,main weight
0,-5.094301e+04,"(2, 15, 0.3)",2,15,0.3
1,-4.910879e+04,"(2, 15, 0.5)",2,15,0.5
2,-4.775033e+04,"(2, 15, 0.7)",2,15,0.7
3,-5.177770e+04,"(2, 30, 0.3)",2,30,0.3
4,-4.944245e+04,"(2, 30, 0.5)",2,30,0.5
5,-4.788089e+04,"(2, 30, 0.7)",2,30,0.7
6,-5.141493e+04,"(2, 50, 0.3)",2,50,0.3
7,-4.909168e+04,"(2, 50, 0.5)",2,50,0.5
8,-4.767621e+04,"(2, 50, 0.7)",2,50,0.7
9,-5.278688e+04,"(5, 15, 0.3)",5,15,0.3


In [135]:
# Plot heatmap with main width on x axis, tail width on y axis and log_p_S_given_D as the color, just use the first 3 main weights

fig = make_subplots(1, 3)


def add_heatmap_for_weight(df, weight, col):
    df = df[df["main weight"] == weight]
    fig.add_trace(
        go.Heatmap(z=df["log_p_S_given_D"], x=df["main width"], y=df["tail width"]),
        row=1,
        col=col,
    )
    fig.update_xaxes(
        type="category",
        row=1,
        col=col,
        title="Main width<br><br>Main weight=" + str(weight),
    )


add_heatmap_for_weight(df_heatmap, 0.3, 1)
add_heatmap_for_weight(df_heatmap, 0.5, 2)
add_heatmap_for_weight(df_heatmap, 0.7, 3)

fig.update_layout(
    title=f"Log probability of S given D for different AR change parameters ({len(IDs.keys())} IDs)",
    yaxis_title="Tail width",
    # yaxis_type='log',
    width=650,
    height=300,
    font_size=10,
)
# Recude font size of the subplot titles
for annotation in fig["layout"]["annotations"]:
    annotation["font"] = dict(size=14)
# use the same color scale for all subplots
fig.update_traces(
    zmin=min(df_heatmap.log_p_S_given_D[df_heatmap.log_p_S_given_D != -np.inf]),
    zmax=max(df_heatmap.log_p_S_given_D),
    colorbar=dict(title="log_p_S_given_D"),
)
# X axis and y axis are categorical
fig.update_yaxes(type="category")

fig.show()

In [ ]:
# The weights is the most differentiating parameter, then main width. Tail width is a smaller effect, althought very important, and remains hidden
# 0.7 weight
# std 2 -> laplace would be better
# tail width doesn't change. Go wider?

# Since the tails is a secondary effet, let's find the rest first

# 3 laplace
# 3 gaussians

In [149]:
params = [
    (0.5, False),
    (0.5, True),
    (1, False),
    (1, True),
    (1.5, False),
    (1.5, True),
    (2, False),
    (2, True),
    (2.5, False),
    (2.5, True),
]
labels = ["main width", "laplace"]

df_10, IDs_10 = load_and_process_json(10, params, labels)
df_10

,log_p_S_given_D,params,main width,laplace
0,-6.655665e+04,"(0.5, False)",0.5,False
1,-6.628876e+04,"(0.5, True)",0.5,True
2,-6.761041e+04,"(1, False)",1.0,False
3,-6.730061e+04,"(1, True)",1.0,True
4,-6.848776e+04,"(1.5, False)",1.5,False
5,-6.834117e+04,"(1.5, True)",1.5,True
6,-6.921651e+04,"(2, False)",2.0,False
7,-6.927346e+04,"(2, True)",2.0,True
8,-inf,"(2.5, False)",2.5,False
9,-inf,"(2.5, True)",2.5,True


In [164]:
# Plot heatmap with main width on x axis, tail width on y axis and log_p_S_given_D as the color, just use the first 3 main weights

tail_width = 30
main_weight = 0.7

fig = make_subplots(1, 1)

fig.add_trace(
    go.Heatmap(z=df_10["log_p_S_given_D"], x=df_10["main width"], y=df_10["laplace"]),
    row=1,
    col=1,
)
fig.update_xaxes(
    type="category",
    row=1,
    col=1,
    title="Main width",
)

fig.update_layout(
    title=f"Log P(S|D) ({len(IDs_10)} IDs)<br>tail width={tail_width}, main weight={main_weight}",
    yaxis_title="Laplace",
    # yaxis_type='log',
    width=400,
    height=250,
    font_size=10,
)
# Recude font size of the subplot titles
for annotation in fig["layout"]["annotations"]:
    annotation["font"] = dict(size=14)
# use the same color scale for all subplots
fig.update_traces(
    zmin=min(df_10.log_p_S_given_D[df_10.log_p_S_given_D != -np.inf]),
    zmax=max(df_10.log_p_S_given_D),
    colorbar=dict(title="log_p_S_given_D"),
)
# X axis and y axis are categorical
fig.update_yaxes(type="category")

fig.show()

In [165]:
mean = 1
sigma_spike = 0.5
weight_spike = 0.7
sigma_tails = [5, 15, 30]
laplace_main = True
laplace_tail = True

params = [5, 15, 30]
labels = ["tail width"]

df_3, IDs_3 = load_and_process_json(3, params, labels)
df_3

,log_p_S_given_D,params,tail width
0,-36042.082673,5,5
1,-36209.401707,15,15
2,-36285.573322,30,30


## Refine weights with best matching std_m, std_t

In [170]:
mean = 1
sigma_spike = 0.2
weight_spike = [
    0.59,
    0.61,
    0.64,
    0.67,
    0.7,
    0.73,
    0.76,
    0.79,
    0.81,
    0.84,
    0.87,
    0.9,
    0.93,
    0.96,
    0.99,
]
sigma_tail = 30
laplace_main = False
laplace_tail = False
params = weight_spike
labels = ["main weight"]

df_15, IDs_15 = load_and_process_json("15_weights", params, labels)

In [171]:
df_15

,log_p_S_given_D,params,main weight
0,-60990.264178,0.59,0.59
1,-60959.335463,0.61,0.61
2,-60923.118143,0.64,0.64
3,-60898.706412,0.67,0.67
4,-60885.942021,0.70,0.70
5,-60885.020931,0.73,0.73
6,-60896.567399,0.76,0.76
7,-60921.781768,0.79,0.79
8,-60947.152258,0.81,0.81
9,-61000.380502,0.84,0.84


In [ ]:
# Plot heatmap with main width on x axis, tail width on y axis and log_p_S_given_D as the color, just use the first 3 main weights

main_width = 5
main_weight = 0.7

fig = make_subplots(1, 1)

fig.add_trace(
    go.Heatmap(z=df_3["log_p_S_given_D"], x=df_3["tail width"], y=df_3["laplace"]),
    row=1,
    col=1,
)
fig.update_xaxes(
    type="category",
    row=1,
    col=1,
    title="Tails width",
)

fig.update_layout(
    title=f"Log P(S|D) ({len(IDs_10)} IDs)<br>main width={main_width}, main weight={main_weight}, all laplace",
    # yaxis_title="Laplace",
    # yaxis_type='log',
    width=400,
    height=200,
    font_size=10,
)
# Recude font size of the subplot titles
for annotation in fig["layout"]["annotations"]:
    annotation["font"] = dict(size=14)
# use the same color scale for all subplots
fig.update_traces(
    zmin=min(df_3.log_p_S_given_D[df_3.log_p_S_given_D != -np.inf]),
    zmax=max(df_3.log_p_S_given_D),
    colorbar=dict(title="log_p_S_given_D"),
)
# X axis and y axis are categorical
fig.update_yaxes(type="category")

fig.show()

## Weight 0.73

In [172]:
# Refine main width

mean = 1
sigma_spike = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
weight_spike = 0.73
sigma_tail = 30
laplace_main = False
laplace_tail = False
params = sigma_spike
labels = ["main std"]

df_9, IDs_9 = load_and_process_json("9_stdmain", params, labels)

In [173]:
df_9

,log_p_S_given_D,params,main std
0,-61008.903745,0.1,0.1
1,-60885.020931,0.2,0.2
2,-60919.984541,0.3,0.3
3,-61017.486311,0.4,0.4
4,-61142.310770,0.5,0.5
5,-61260.143180,0.6,0.6
6,-61372.719100,0.7,0.7
7,-61521.470012,0.8,0.8
8,-61708.376343,0.9,0.9


## Weight 0.73, main std 0.2

In [178]:
mean = 1
sigma_spike = 0.2
weight_spike = 0.73
sigma_tail = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70]
laplace_main = False
laplace_tail = False
params = sigma_tail
print(len(params))

df_tail, IDs_tail = load_and_process_json("15_stdtail", params, labels)

15
